# Showing Selective Disclosure with CL signature

Code is used to show the mathematics details for ZKP-CL.

The CL-signature scheme defines the public key in this key pair as the quadruple $(n, Z, S ,\{R_i\}_{i \in A})$ where $A$ is the set of indices of attributes in the credential schema, $n$ is a Special RSA Modulus, and $Z, S , {R_i}$ are random quadratic residues modulo n.

The public key information $(n, Z, S ,\{R_i\}_{i \in A})$ are publicly available on the ledger, the signature $(A, e, v)$ can only be calculated by the issuer, who owns the private key $(p, q)$.

## Equations


Remember from the math section:

$$ T = A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $$
$$ A' = AS^r $$
$$ v' = v - er $$
$$ \hat{v} = \bar{v} + Dv'  $$
$$ e' = e - \beta $$
$$ \hat{e} = \bar{e} + De' $$
$$ \hat{m_i} = \bar{m_i} + Dm_i $$
$$ Q = A^e \pmod{n} $$
$$ Q = \Bigg(\frac{Z}{S^v \prod_{i \in A}{R_i^{m_i}}}\Bigg) \pmod{n} $$
$$ \hat{T} = \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n} $$

Check CL signature with equation

$$
\begin{align*}
Q &= A^e \\
A^e &=\Bigg(\frac{Z}{S^v \prod_{i \in A}{R_i^{m_i}}}\Bigg) \\
Z &= A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) \\

\end{align*}
$$

In [1]:
from Crypto.Util import number

In [2]:
# For demo purposes to make print more readable and consistent
bitsize = 32
p = 3397833353
q = 2516557159
# p = number.getPrime(bitsize)
# q = number.getPrime(bitsize)

# The public parts
n = p*q
nsize = 2*bitsize
# Generate quadratic residues mod n
R_0 = pow(number.getRandomRange(2, n), 2, n)
R_1 = pow(number.getRandomRange(2, n), 2, n)
R_2 = pow(number.getRandomRange(2, n), 2, n)
R_3 = pow(number.getRandomRange(2, n), 2, n)
R_4 = pow(number.getRandomRange(2, n), 2, n)
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)

# The messages
msize = nsize
m_0 = number.getRandomRange(2, n)
m_1 = number.getRandomRange(2, n)
m_2 = number.getRandomRange(2, n)
m_3 = number.getRandomRange(2, n)
m_4 = number.getRandomRange(2, n)


# The signature parts
e = number.getPrime(nsize + 1)
e_inv = pow(e, -1, (p-1)*(q-1))
# v is selected as integer
v = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate dot
R_vector = pow(R_0, m_0, n) * pow(R_1, m_1, n) * \
    pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_4, m_4, n)

commitment_vector = pow(S, v, n) * R_vector
commitment_vector_inv = pow(commitment_vector, -1, n)

# Calculate Q
Q = (Z * commitment_vector_inv) % n
# Calculate signature part A
A = pow(Q, e_inv, n)

print(f'Public key is:\nn = {n}\n(R_0, R_1, R_2, R_3, R_4) = {(R_0, R_1, R_2, R_3, R_4)}\nS = {S}\nZ = {Z}')
print(f'\nPrivate key is: {(p,q)}')
print(f'\nMessage (m_0, m_1, m_2, m_3, m_4) is ({m_0}, {m_1}) \nSignature on (m_0, m_1, m_2, m_3, m_4) is: e = {e}, s = {A}, v = {v}')

# verify credentials
print('\nRHS is the same as LHS:', pow(A, e, n) == Q)
print('RHS is the same as LHS:', (pow(A, e, n) * commitment_vector) % n == Z)


Public key is:
n = 8550841849581124127
(R_0, R_1, R_2, R_3, R_4) = (5710887662410021578, 5131450540407428543, 273761181316097119, 7451183371823971335, 789372916845624975)
S = 5296238720553891307
Z = 1487901499195261306

Private key is: (3397833353, 2516557159)

Message (m_0, m_1, m_2, m_3, m_4) is (264788961767077034, 1631490665936586445) 
Signature on (m_0, m_1, m_2, m_3, m_4) is: e = 33301153974451987313, s = 4762865503745813998, v = 16274418766789697532958677015973131518773030350508847671126894583677149705938416344589204391279322651092824337871058337123632703441296372568776268139770893735418723863390070396347210922192566347873342117392758257021209647151949023093016115649604115832195682317166883514431629402354111064734104613471482832741641925309868965066554632245773938859039569436380912480472138560365506095939599258250814216114953940635401173795539765443716665406715580978688846479562352656708169203993588289485326989062246353495903111573322903853988004217099475640498468959479074744918

# Why selective disclosure works


Let A be the set of all attribute identifiers present in the credential schema

where $A = A_r \cup A_h$

such that $A_r$ contains revealed attributes to the Verifier and $A_h$ contains unrevealed (hidden) attributes which are kept secret.
Since $A_r$ and $A_h$ are mutually exclusive, we know that

$ \prod_{i \in A}{R_i^{m_i}} = \prod_{i \in A_r}{R_i^{m_i}} \cdot \prod_{i \in A_h}{R_i^{m_i}}$


This allows us to prove that a credential contains a set of attributes with the revealed values $\{m_i\}_{i \in A_r}$
by proving knowledge of the exponents $(e, \{m_i\}_{i \in A_h}, v)$ with Equation

$$
\begin{align*}
Z &= A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) \\
Z &= A^e \cdot S^v \cdot \Bigg(\prod_{i \in A_r}{R_i^{m_i}} \cdot \prod_{i \in A_h}{R_i^{m_i}}\Bigg) \\
\Bigg(\frac{Z}{\prod_{i \in A_r}{R_i^{m_i}}}\Bigg) &= A^e \cdot S^v \cdot \Bigg(\prod_{i \in A_h}{R_i^{m_i}}\Bigg) \\
Z \Bigg(\prod_{i \in A_r}{R_i^{m_i}}\Bigg)^{-1} &= A^e S^v \Bigg(\prod_{i \in A_h}{R_i^{m_i}}\Bigg)

\end{align*}
$$

In [3]:
# private attributes
R_private = pow(R_0, m_0, n) * pow(R_3, m_3, n) * pow(R_4, m_4, n)
# public attributes
R_public = pow(R_1, m_1, n) * pow(R_2, m_2, n)

# generate commitment and encryption
R_public_inv = pow(R_public, -1, n)
# contains revealed attributes
attr_encryption_vector = Z * R_public_inv

# contains unrevealed attributes which are kept secret
attr_commitment_vector = pow(S, v, n) * R_private

# verify credentials
print('\nRHS is the same as LHS:', (pow(A, e, n) * attr_commitment_vector) % n == attr_encryption_vector % n)



RHS is the same as LHS: True


## Reference
- https://github.com/PeterAltmann/privacy_techniques/blob/main/ZKP-CL-SRSA.ipynb
- https://github.com/PeterAltmann/privacy_techniques/blob/e98595a869a98320a3d66db35b341d62d7d81f1f/math_CL.ipynb
